In [ ]:
import pandas as pd
import numpy as np
import os
import json
import pyodbc
import csv

from etl.load_data import create_access_token

## Retrieve most recent data

In [ ]:
def get_data(sql, config):
    conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=' + config['db_host'] +';DATABASE=' + config['db_name'] +';UID='+config['db_user'] +';PWD='+ config['db_password'] )
    cursor = conn.cursor()

    cursor.execute(sql)

    result = cursor.fetchall()

    headers = [column[0] for column in cursor.description]
    
    cursor.close()
    conn.close()
    
    return result, headers

In [ ]:
with open('etl/config.json') as json_data:
    config = json.load(json_data)

with open("sql/get_data.sql", "r") as f:
    sql = f.read()

In [ ]:
result, headers = get_data(sql, config)

In [ ]:
with open('auction_data.csv', 'w', newline='') as out:
    csv_out = csv.writer(out)
    csv_out.writerow(headers)
    for row in result:
        csv_out.writerow(row)